## Imports

In [1]:
# Default
import sys
import datetime

sys.path.append('../src')
sys.path.append('../')
sys.path.append('../../da-irl')

import numpy as np
from impl.activity_mdp import ATPTransition, ActivityMDP
from impl.activity_rewards import ActivityRewardFunction
from impl.persona_population_data import ExpertPersonaAgent
from irl.maxent_irl import MaxEntIRL



from file_io.activity_config import ATPConfig
from impl.activity_env import ActivityEnv
from misc.math_utils import create_dir_if_not_exists

# Vendor
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from pandas.tseries.offsets import Day
import shapely
from shapely.geometry import box
import numpy as np
import folium

shapely.speedups.enable()



In [ ]:
20

In [2]:
from swlcommon import TraceLoader

In [3]:
# Global magics statements

%load_ext autoreload
%autoreload 2
%matplotlib inline

## Constants and Utility Functions

In [4]:
# BBOX:
LAT_MIN=32.831
LAT_MAX=38.67
LNG_MIN=-128.9
LNG_MAX=-117.41

# DOW
MON = 0
TUES = 1
WEDS = 2
THUR = 3
FRI = 4
SAT = 5
SUN = 6

# Path
PATH = '../../da-irl/data/traces/traces_persona_2.csv'

def great_circle_distance(pt1,pt2):
    """
    Return the great-circle distance in kilometers between two points,
    defined by a tuple (lat, lon).
    Examples
    --------
    >>> brussels = (50.8503, 4.3517)
    >>> paris = (48.8566, 2.3522)
    >>> great_circle_distance(brussels, paris)
    263.9754164080347
    """
    r = 6371.
    
    delta_latitude = np.radians(pt1[:,0] - pt2[:,0])
    delta_longitude = np.radians(pt1[:,1] - pt2[:,1])
    latitude1 = np.radians(pt1[:,0])
    latitude2 = np.radians(pt2[:,0])

    a = np.sin(delta_latitude / 2) ** 2 + np.cos(latitude1) * np.cos(latitude2) * np.sin(delta_longitude / 2) ** 2
    return r * 2. * np.arcsin(np.sqrt(a))


## Domain

In [1119]:
#         self._records = gpd.GeoDataFrame(df.drop(['lat', 'lng'], axis=1),
#                                 crs={'init': 'epsg:4326'},
#                                 geometry=[shapely.geometry.Point(xy) for xy in zip(df.lat, df.lng)])

In [ ]:
def get_legal_actions_for_state(self, el):
    """
    If in an activity state, the agent can either stay in the activity or transition to a travel state
    (any available mode)

     If in a travelling state, the agent can either stay in the current mode, transition to a different mode,
      or arrive at an activity (essentially, any choice of action is available).

    Args:
        el: the current state type of the agent.

    Returns:
        A list of the available legal actions for the current state.

    """
    if el in self.activity_types:
        q = [k for k, v in self.actions.items() if (v.succ_ix in self.travel_modes or v.succ_ix == el)]
    elif el in self.travel_modes:
        q = self.actions.keys()
    else:
        raise ValueError("%s not Found!" % el)
    return q

def _build_actions(self):
    actions = {}
    for action_ix, el in enumerate(self.activity_types + self.travel_modes):
        actions[action_ix] = ATPAction(action_ix, el)
        action_ix += 1
    return actions

def build_state_graph():
    """
    Builds the mdp state graph from the travel plans

    Returns:
        The state graph
    """

    # tmat = data.tmat  # SxT matrix

    g = nx.DiGraph()
    state_ix = 0

    # Add unique states
    term = False
    for t in range(self.horizon):
        for s, el in enumerate(self.activity_types + self.travel_modes):
            edge = (el, t)
            if t < self.horizon - 1:  # if it's not the last period, we can still make decisions
                available_actions = self.get_legal_actions_for_state(el)
            else:
                available_actions = [self.get_home_action_id()]
                term = True
            for a in available_actions:
                ns_el = self.actions[a].succ_ix
                g.add_edge(edge, (ns_el, t + 1), attr_dict={'ix': a})
            if el in self.activity_types:
                el_type = ActivityState(state_ix, el, t, self.segment_mins, edge)
            elif el in self.travel_modes:
                el_type = TravelState(state_ix, el, t, self.segment_mins, edge)
            else:
                raise ValueError("%s not Found!" % el)
            if term:
                el_type = ActivityState(state_ix, self.home_act, t, self.segment_mins, edge)
                self.terminals.append(state_ix)
                self.home_state = el_type
                g.add_edge(edge, (self.home_act, t), attr_dict={'ix': available_actions[0]})
            el_type.available_actions.extend(available_actions)
            g.add_node(edge, attr_dict={'ix': state_ix, 'pos': edge, 'state': el_type})
            self.states[state_ix] = el_type
            state_ix += 1
    return g

In [2]:
import tensorflow as tf

In [ ]:
sess = tf.InteractiveSession()

In [46]:
p = [0.1, 0.5, 0.4]
c = tf.contrib.distributions.Categorical(p)


ImportError: cannot import name bayesflow

In [6]:
import networkx as nx


g = nx.DiGraph()

